# Chapter 11 統計的仮説検定

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
%precision 3
np.random.seed(1111)

In [2]:
df = pd.read_csv('../data/ch11_potato.csv')

sample = np.array(df['重さ'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [3]:
s_mean = np.mean(sample)
s_mean

128.4507142857143

## 11.1 |　統計的仮説検定とは

### 11.1.1　統計的仮説検定の基本

In [4]:
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95)

128.68118313069039

In [5]:
z = (s_mean -130)/np.sqrt(9/14)
z

-1.932298779026813

In [6]:
rv = stats.norm()
rv.isf(0.95)

-1.6448536269514722

In [7]:
# p 値
rv.cdf(z)

0.026661319523126635

### 11.1.2　片側検定と両側検定

In [8]:
z = (s_mean - 130)/np.sqrt(9/14)
z

-1.932298779026813

In [9]:
rv = stats.norm()
rv.interval(0.95)

(-1.959963984540054, 1.959963984540054)

片側検定の方が両側検定より帰無仮説を棄却しやすい

In [10]:
rv.cdf(z) * 2

0.05332263904625327

### 11.1.3　仮説検定における 2つの過誤

__第一種の過誤__  帰無仮説が正しいときに, 帰無仮説を棄却してしまう過誤  
__第二種の過誤__  対立仮説が正しいときに, 帰無仮説を採択してしまう過誤

In [11]:
rv = stats.norm(130, 3)

In [12]:
c = stats.norm().isf(0.95)
n_sample = 10000
cnt = 0

for _ in range(n_sample):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130)/np.sqrt(9/14)
    
    if z < c:
        cnt += 1
        
cnt/n_sample

0.053

In [13]:
rv = stats.norm(128, 3)

In [14]:
c = stats.norm().isf(0.95)
n_sample = 10000
cnt = 0

for _ in range(n_sample):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130)/np.sqrt(9/14)
    
    if z >= c:
        cnt += 1
        
cnt/n_sample

0.197

## 11.2 |　基本的な仮説検定

### 11.2.1　正規分布の母平均の検定 (母分散概知)

In [25]:
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)
    
    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
         print('Accept the null hypothesis')
    else:
         print('Reject the null hypothesis')
        
    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1 - rv.cdf(z)) * 2
    print(f'p-value: {p:.3f}')

In [26]:
pmean_test(sample, 130, 9)

Accept the null hypothesis
p-value: 0.053


### 11.2.2 正規分布の母分散の検定

In [27]:
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('Accept the null hypothesis')
    else:
        print('Reject the null hypothesis')
        
    if y < 0:
        p = rv.cdf(y) * 2
    else:
        p = (1 - rv.cdf(y)) * 2
    print(f'p-value: {p:.3f}')

In [28]:
pvar_test(sample, 9)

Accept the null hypothesis
p-value: 0.085


### 11.2.3　正規分布の母平均の検定 (母分散未知)

In [29]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)
    
    t = (s_mean -mean0) / np.sqrt(u_var/n)
    if interval[0] <= t <= interval[1]:
        print('Accept the null hypothesis')
    else:
        print('Reject the null hypothesis')
        
    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p-value: {p:.3f}')

In [30]:
pmean_test(sample, 130)

Accept the null hypothesis
p-value: 0.169


In [31]:
t, p = stats.ttest_1samp(sample, 130)
t, p

(-1.4551960206404198, 0.16933464230414275)